In [1]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -qq langdetect
from langdetect import detect
import string
!pip install -qq contractions
!pip install -qq demoji
import demoji
demoji.download_codes()
!pip install -qq datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 722.0 kB/s eta 0:00:00


<ipython-input-2-28a5514cb77a>:7: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00


In [3]:
header_names = ['label', 'text']
df_train = pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/kannada_offensive.csv", sep=',',names=header_names)
df_train = df_train.dropna(how='all')
df_train = pd.DataFrame(df_train)
df_train

,label,text
0,not-Kannada,@Kavya V u wanna go there?
1,Not_offensive,@poojitha subbaiah Hai ennane ullira
2,Offensive_Untargetede,Namma galli goo bandiddru ashwatthama
3,not-Kannada,Hahaha super bro
4,not-Kannada,@Saraswathi.p.k. Baby swathi
...,...,...
7767,not-Kannada,@manz game yes!
7768,not-Kannada,anup kulkarni... True
7769,Not_offensive,ಇದು ಚರಿತ್ರೆ ಸೃಷ್ಟಿಸೊ ಅವತಾರಾ..........
7770,Not_offensive,Idu cheritre shrustiso avatara


In [4]:
label_mapping = {
    'Not_offensive':0,
    'Offensive_Untargetede':1,
    'Offensive_Targeted_Insult_Individual':1,
    'Offensive_Targeted_Insult_Group':1,
    'Offensive_Targeted_Insult_Other':1,
    'not-Kannada':2,
}
df_train['label'].replace(label_mapping, inplace=True)
df_train = df_train[df_train['label'] != 2]
print(df_train.info())
label_counts = df_train['label'].value_counts()
print(label_counts)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5874 entries, 1 to 7771
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5874 non-null   int64 
 1   text    5874 non-null   object
dtypes: int64(1), object(1)
memory usage: 137.7+ KB
None
0    4397
1    1477
Name: label, dtype: int64


In [5]:
df_train.head(20)

,label,text
1,0,@poojitha subbaiah Hai ennane ullira
2,1,Namma galli goo bandiddru ashwatthama
5,0,Ella hantha hellodakke hagodilla eddaru erabha...
6,0,Satyana edu devare
7,0,ನಂಬಲಾದ ವಿಸ್ಮಯ ಸಂಗತಿಗಳು ಅದೆಷ್ಟು ಇವೆ ನಂಬಲು ಆಗದೆ ...
8,0,Kathe tumba ista aythu..
9,0,ಗುರು ಇದನ್ನು ಮಾಧ್ಯಮದವರು ಸರೆ ಹಿಡ್ದಿರೋ ದೃಶ್ಯ ಇದ್ರ...
10,0,Howdu brother nanu tv nalli nodidde evru elidn...
11,0,ಕಿರಣ್ ಶಾಲಿ... ಇಲ್ಲಿ ಕೊಟ್ಟಿದೀನಿ ನೋಡು ಈ ಲಿಂಕ್ ಕ...
12,1,@vinay vinu ಅಶ್ವತ್ಥಾಮ ಹೇಗೆ ಚಿರಂಜೀವಿಯೋ ಹಾಗೆಯೇ ಭ...


In [6]:
import sys
sys.path.append("/content/drive/MyDrive/Capstone/Translator/")
from google_trans_new import google_translator

In [7]:
translator = google_translator()
for index, row in df_train.iterrows(): # Check if emoji is contained in text
    text = str(row['text'])
    found = demoji.findall(text)
    if found:
        # There is an emoji now detect language
        if detect(text) == 'kn': # Text is Kannada
            for key, value in found.items():
                emojiEng = value
                # Translate to Kannada
                transText = translator.translate(emojiEng, lang_tgt='kn', lang_src='en')
                text = text.replace(key, transText + ' ')
        else: # Text is english
            for key, value in found.items():
                text = text.replace(key, value + ' ')

        df_train.at[index, 'text'] = text

In [8]:
df_train.head(30)

,label,text
1,0,@poojitha subbaiah Hai ennane ullira
2,1,Namma galli goo bandiddru ashwatthama
5,0,Ella hantha hellodakke hagodilla eddaru erabha...
6,0,Satyana edu devare
7,0,ನಂಬಲಾದ ವಿಸ್ಮಯ ಸಂಗತಿಗಳು ಅದೆಷ್ಟು ಇವೆ ನಂಬಲು ಆಗದೆ ...
8,0,Kathe tumba ista aythu..
9,0,ಗುರು ಇದನ್ನು ಮಾಧ್ಯಮದವರು ಸರೆ ಹಿಡ್ದಿರೋ ದೃಶ್ಯ ಇದ್ರ...
10,0,Howdu brother nanu tv nalli nodidde evru elidn...
11,0,ಕಿರಣ್ ಶಾಲಿ... ಇಲ್ಲಿ ಕೊಟ್ಟಿದೀನಿ ನೋಡು ಈ ಲಿಂಕ್ ಕ...
12,1,@vinay vinu ಅಶ್ವತ್ಥಾಮ ಹೇಗೆ ಚಿರಂಜೀವಿಯೋ ಹಾಗೆಯೇ ಭ...


In [9]:
for index, row in df_train.iterrows():
    found = demoji.findall(row['text'])
    if found:
        print('Found exception')

In [10]:
import contractions
# contracted text
for index, row in df_train.iterrows():# creating an empty list
    expanded_words = []
    for word in str(row['text']).split():# using contractions.fix to expand the shotened words
        expanded_words.append(contractions.fix(word))

    expanded_text = ' '.join(expanded_words)
    df_train.at[index, 'text'] = expanded_text


for index, row in df_train.iterrows():
    Sentence = str(row['text'])
    for i in Sentence:
        # checking whether the char is punctuation.
        if i in string.punctuation:
            Sentence = Sentence.replace(i, '')
    df_train.at[index, 'text'] = Sentence.lower()

In [11]:
df_train.head(30)

,label,text
1,0,poojitha subbaiah hai ennane ullira
2,1,namma galli goo bandiddru ashwatthama
5,0,ella hantha hellodakke hagodilla eddaru erabha...
6,0,satyana edu devare
7,0,ನಂಬಲಾದ ವಿಸ್ಮಯ ಸಂಗತಿಗಳು ಅದೆಷ್ಟು ಇವೆ ನಂಬಲು ಆಗದೆ ...
8,0,kathe tumba ista aythu
9,0,ಗುರು ಇದನ್ನು ಮಾಧ್ಯಮದವರು ಸರೆ ಹಿಡ್ದಿರೋ ದೃಶ್ಯ ಇದ್ರ...
10,0,howdu brother nanu tv nalli nodidde evru elidn...
11,0,ಕಿರಣ್ ಶಾಲಿ ಇಲ್ಲಿ ಕೊಟ್ಟಿದೀನಿ ನೋಡು ಈ ಲಿಂಕ್ ಕ್ಲಿಕ...
12,1,vinay vinu ಅಶ್ವತ್ಥಾಮ ಹೇಗೆ ಚಿರಂಜೀವಿಯೋ ಹಾಗೆಯೇ ಭಗ...


In [12]:
kannada_stopwords = [
    'ಮತ್ತು', 'ಈ', 'ಒಂದು', 'ರಲ್ಲಿ', 'ಹಾಗೂ', 'ಎಂದು', 'ಅಥವಾ', 'ಇದು', 'ರ', 'ಅವರು', 'ಎಂಬ',
    'ಮೇಲೆ', 'ಅವರ', 'ತನ್ನ', 'ಆದರೆ', 'ತಮ್ಮ', 'ನಂತರ', 'ಮೂಲಕ', 'ಹೆಚ್ಚು', 'ನ', 'ಆ', 'ಕೆಲವು',
    'ಅನೇಕ', 'ಎರಡು', 'ಹಾಗು', 'ಪ್ರಮುಖ', 'ಇದನ್ನು', 'ಇದರ', 'ಸುಮಾರು', 'ಅದರ', 'ಅದು', 'ಮೊದಲ',
    'ಬಗ್ಗೆ', 'ನಲ್ಲಿ', 'ರಂದು', 'ಇತರ', 'ಅತ್ಯಂತ', 'ಹೆಚ್ಚಿನ', 'ಸಹ', 'ಸಾಮಾನ್ಯವಾಗಿ', 'ನೇ',
    'ಹಲವಾರು', 'ಹೊಸ', 'ದಿ', 'ಕಡಿಮೆ', 'ಯಾವುದೇ', 'ಹೊಂದಿದೆ', 'ದೊಡ್ಡ', 'ಅನ್ನು', 'ಇವರು',
    'ಪ್ರಕಾರ', 'ಇದೆ', 'ಮಾತ್ರ', 'ಕೂಡ', 'ಇಲ್ಲಿ', 'ಎಲ್ಲಾ', 'ವಿವಿಧ', 'ಅದನ್ನು', 'ಹಲವು', 'ರಿಂದ',
    'ಕೇವಲ', 'ದ', 'ದಕ್ಷಿಣ', 'ಗೆ', 'ಅವನ', 'ಅತಿ', 'ನೆಯ', 'ಬಹಳ', 'ಕೆಲಸ', 'ಎಲ್ಲ', 'ಪ್ರತಿ',
    'ಇತ್ಯಾದಿ', 'ಇವು', 'ಬೇರೆ', 'ಹೀಗೆ', 'ನಡುವೆ', 'ಇದಕ್ಕೆ', 'ಎಸ್', 'ಇವರ', 'ಮೊದಲು', 'ಶ್ರೀ',
    'ಮಾಡುವ', 'ಇದರಲ್ಲಿ', 'ರೀತಿಯ', 'ಮಾಡಿದ', 'ಕಾಲ', 'ಅಲ್ಲಿ', 'ಮಾಡಲು', 'ಅದೇ', 'ಈಗ', 'ಅವು',
    'ಗಳು', 'ಎ', 'ಎಂಬುದು', 'ಅವನು', 'ಅಂದರೆ', 'ಅವರಿಗೆ', 'ಇರುವ', 'ವಿಶೇಷ', 'ಮುಂದೆ', 'ಅವುಗಳ',
    'ಮುಂತಾದ', 'ಮೂಲ', 'ಬಿ', 'ಮೀ', 'ಒಂದೇ', 'ಇನ್ನೂ', 'ಹೆಚ್ಚಾಗಿ', 'ಮಾಡಿ', 'ಅವರನ್ನು', 'ಇದೇ',
    'ಯ', 'ರೀತಿಯಲ್ಲಿ', 'ಜೊತೆ', 'ಅದರಲ್ಲಿ', 'ಮಾಡಿದರು', 'ನಡೆದ', 'ಆಗ', 'ಮತ್ತೆ', 'ಪೂರ್ವ', 'ಆತ',
    'ಬಂದ', 'ಯಾವ', 'ಒಟ್ಟು', 'ಇತರೆ', 'ಹಿಂದೆ', 'ಪ್ರಮಾಣದ', 'ಗಳನ್ನು', 'ಕುರಿತು', 'ಯು', 'ಆದ್ದರಿಂದ',
    'ಅಲ್ಲದೆ', 'ನಗರದ', 'ಮೇಲಿನ', 'ಏಕೆಂದರೆ', 'ರಷ್ಟು', 'ಎಂಬುದನ್ನು', 'ಬಾರಿ', 'ಎಂದರೆ', 'ಹಿಂದಿನ',
    'ಆದರೂ', 'ಆದ', 'ಸಂಬಂಧಿಸಿದ', 'ಮತ್ತೊಂದು', 'ಸಿ', 'ಆತನ'
]


In [13]:
data = []

for index, row in df_train.iterrows():
    sent = ''
    line = str(row['text'])
    for i in line.split():
        if i not in kannada_stopwords:
            sent += i + " "
    data.append({'text': sent, 'label': str(row['label'])})

new_df_train = pd.DataFrame(data)
new_df_train.head(30)


,text,label
0,poojitha subbaiah hai ennane ullira,0
1,namma galli goo bandiddru ashwatthama,1
2,ella hantha hellodakke hagodilla eddaru erabha...,0
3,satyana edu devare,0
4,ನಂಬಲಾದ ವಿಸ್ಮಯ ಸಂಗತಿಗಳು ಅದೆಷ್ಟು ಇವೆ ನಂಬಲು ಆಗದೆ ...,0
5,kathe tumba ista aythu,0
6,ಗುರು ಮಾಧ್ಯಮದವರು ಸರೆ ಹಿಡ್ದಿರೋ ದೃಶ್ಯ ಇದ್ರೆ ಹಾಕು ...,0
7,howdu brother nanu tv nalli nodidde evru elidn...,0
8,ಕಿರಣ್ ಶಾಲಿ ಕೊಟ್ಟಿದೀನಿ ನೋಡು ಲಿಂಕ್ ಕ್ಲಿಕ್ ಮಾಡು,0
9,vinay vinu ಅಶ್ವತ್ಥಾಮ ಹೇಗೆ ಚಿರಂಜೀವಿಯೋ ಹಾಗೆಯೇ ಭಗ...,1


In [14]:
new_df_train['text'].isnull().sum()

0

In [17]:
!pip install -qq indic-nlp-library

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 17.2 MB/s eta 0:00:00


In [18]:
from indicnlp import common
from indicnlp import loader
from indicnlp.script import indic_scripts
from indicnlp.tokenize import indic_tokenize
from indicnlp.morph import unsupervised_morph

In [16]:
!git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

Cloning into 'indic_nlp_resources'...
remote: Enumerating objects: 139, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 139 (delta 2), reused 2 (delta 0), pack-reused 126
Receiving objects: 100% (139/139), 149.77 MiB | 35.16 MiB/s, done.
Resolving deltas: 100% (53/53), done.
Updating files: 100% (28/28), done.


In [19]:
INDIC_NLP_RESOURCES = r"/content/indic_nlp_resources"
common.set_resources_path(INDIC_NLP_RESOURCES)
loader.load()
analyzer = unsupervised_morph.UnsupervisedMorphAnalyzer('kn')

In [20]:
from langdetect import detect, LangDetectException

In [22]:
from tqdm import tqdm

def stem_kannada_text(text):
    tokens = text.split()
    stemmed_tokens = []
    for token in tokens:
        analyzed_tokens = analyzer.morph_analyze_document([token])
        if analyzed_tokens:
            stemmed_tokens.append(analyzed_tokens[0])
        else:
            stemmed_tokens.append(token)  # If stemming fails, keep the original token
    return ' '.join(stemmed_tokens)

# Assuming new_df_train is your DataFrame containing text data
# Add tqdm to the loop to track progress
for index, row in tqdm(new_df_train.iterrows(), total=len(new_df_train)):
    text = str(row['text'])

    try:
        detected_lang = detect(text)
    except LangDetectException as e:
        # Handle the exception, for example by setting the detected_lang to None
        detected_lang = None

    if detected_lang == 'kn':
        stemmed_text = stem_kannada_text(text)
        new_df_train.at[index, 'text'] = stemmed_text


print(new_df_train.head(20))


100%|██████████| 5874/5874 [01:03<00:00, 92.38it/s]

                                                 text label
0                poojitha subbaiah hai ennane ullira      0
1              namma galli goo bandiddru ashwatthama      1
2   ella hantha hellodakke hagodilla eddaru erabha...     0
3                                 satyana edu devare      0
4   ನಂಬ ವಿಸ್ಮಯ ಸಂಗತಿಗಳ ಅದೆಷ್ಟ ಇವೆ ನಂಬ ಆಗ ಇದ್ದ ಸತ್ಯ ಆಗ     0
5                             kathe tumba ista aythu      0
6         ಗುರು ಮಾಧ್ಯಮ ಸರೆ ಹಿಡ್ದ ದೃಶ್ಯ ಇದ್ರೆ ಹಾಕು ಗುರು     0
7   howdu brother nanu tv nalli nodidde evru elidn...     0
8             ಕಿರಣ್ ಶಾಲಿ ಕೊಟ್ಟ ನೋಡು ಲಿಂಕ್ ಕ್ಲಿಕ್ ಮಾಡು     0
9   vinay vinu ಅಶ್ವತ್ಥಾಮ ಹೇಗೆ ಚಿರಂಜೀವಿಯ ಹಾಗೆ ಭಗವಾನ...     1
10  kiran shali ಲೈ ತಗಡು ನೀನು ಹಿಂದು ಧರ್ಮ ಆಥವ cross ...     1
11           vinay vinu ಇವ್ನ ಯಾರ್ ಗುರು ನಂದೆ ಇಡ್ಲಿ ಅಂತ     1
12  kiran shali ಲೈ ಶ್ಯಾ ನಿನ್ ಅಯ್ಯ ಪ್ರಶ್ನೆ ಕೆಳ್ ನೀನ...     1
13  vinay vinu nice comedy man ನಾನ್ ಕೇಳೋ ಪ್ರಶ್ನೆ ಉ...     1
14                                      ಗುರು ನಿಮ್ ಡೌಟ     1
15                 bro z newshindi chann

In [23]:
new_df_train.to_csv('kannada_dataset_2_train.csv', index=False)

In [ ]:
#df=pd.read_csv("/kaggle/working/kannada_dataset_2_train.csv")
#df=pd.DataFrame(df)
#df.head(10)

,text,label
0,kavya v you want to go there,not-Kannada
1,poojitha subbaiah hai ennane ullira,Not_offensive
2,namma galli goo bandiddru ashwatthama,Offensive_Untargetede
3,hahaha super bro,not-Kannada
4,saraswathipk baby swathi,not-Kannada
5,ella hantha hellodakke hagodilla eddaru erabha...,Not_offensive
6,satyana edu devare,Not_offensive
7,ನಂಬಲಾದ ವಿಸ್ಮಯ ಸಂಗತಿಗಳು ಅದೆಷ್ಟು ಇವೆ ನಂಬಲು ಆಗದೆ ...,Not_offensive
8,kathe tumba ista aythu,Not_offensive
9,ಗುರು ಮಾಧ್ಯಮದವರು ಸರೆ ಹಿಡ್ದಿರೋ ದೃಶ್ಯ ಇದ್ರೆ ಹಾಕು ...,Not_offensive
